<a href="https://colab.research.google.com/github/root-git/stratascratch-sql-challenges/blob/main/7_Minimum_Number_of_Platforms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Minimum Number of Platforms

You are given a day worth of scheduled departure and arrival times of trains at one train station. One platform can only accommodate one train from the beginning of the minute it's scheduled to arrive until the end of the minute it's scheduled to depart. Find the minimum number of platforms necessary to accommodate the entire scheduled traffic.

**Original Question Link:**  
[StrataScratch ID 2082 – Minimum Number of Platforms](https://platform.stratascratch.com/coding/2082-minimum-number-of-platforms?code_type=1)

---

## Table Schema

### `train_arrivals`
| Column        | Type   |
|---------------|--------|
| train_id      | bigint |
| arrival_time  | text   |

### `train_departures`
| Column        | Type   |
|---------------|--------|
| train_id      | bigint |
| departure_time| text   |

---

## Thought Process

1. Join arrival and departure tables on `train_id`.
2. Generate all minutes between arrival and departure for each train (inclusive).
3. Flatten the data so each row represents a train occupying a specific minute.
4. Count the number of trains per minute (i.e., concurrent platform use).
5. Take the maximum count. Thisi is the minimum number of platforms required.

In [7]:
import pandas as pd

#Create mock data
arrivals_data = pd.DataFrame({
    'train_id': [1, 2, 3, 4, 5, 6],
    'arrival_time': ['2023-01-01 10:00', '2023-01-01 10:05', '2023-01-01 10:10', '2023-01-01 10:10', '2023-01-01 10:15', '2023-01-01 10:20']
})
departures_data = pd.DataFrame({
    'train_id': [1, 2, 3, 4, 5, 6],
    'departure_time': ['2023-01-01 10:30', '2023-01-01 10:15', '2023-01-01 10:20', '2023-01-01 10:25', '2023-01-01 10:30', '2023-01-01 10:40']
})


In [9]:
import sqlite3

# Load into SQLite (in-memory)
conn = sqlite3.connect(':memory:')
arrivals_data.to_sql("train_arrivals", conn, index=False, if_exists='replace')
departures_data.to_sql("train_departures", conn, index=False, if_exists='replace')

#Show Preview
print(pd.read_sql('SELECT * FROM train_arrivals', conn))

   train_id      arrival_time
0         1  2023-01-01 10:00
1         2  2023-01-01 10:05
2         3  2023-01-01 10:10
3         4  2023-01-01 10:10
4         5  2023-01-01 10:15
5         6  2023-01-01 10:20


In [10]:
# Replace with your SQL query below
query = """ SELECT * FROM train_arrivals"""

result_df = pd.read_sql(query, conn)

In [15]:
query = """
WITH timeline AS
(
SELECT
  arrival_time AS time,
  1 AS change
FROM train_arrivals
UNION ALL
SELECT
  DATETIME(departure_time, '+1 minute')  AS time,
  -1 AS change
FROM train_departures
),
running_sum AS
(
  SELECT
    time,
    SUM(change) OVER (ORDER BY time) AS active_trains
  FROM timeline
)
SELECT
  MAX(active_trains) AS min_platforms
FROM running_sum
"""
solution = pd.read_sql(query, conn)
print(solution)

   min_platforms
0              5


## Problem Explanation

### Step 1: Combine arrival and departure events into one timeline
```sql
SELECT
  arrival_time AS time,
  1 AS change
FROM train_arrivals
UNION ALL
SELECT
  DATETIME(departure_time, '+1 minute') AS time,
  -1 AS change
FROM train_departures
```
- `arrival_time` means +1 train on a platform at that minute.
- `departure_time` +1 minute means the platform becomes free after the departure minute, so we use -1 starting the next minute.

### Step 2: Getting the sum of active trains
```sql
SELECT
  time,
  SUM(change) OVER(ORDER BY time) AS active_trains
FROM timeline
```
- Use `SUM` window function to compute the running sum of trains occupying platforms over time.
- For each minute in the timeline, how many trains are on the platforms at that exact time.

### Step 3: Maximum concurrent trains
```sql
SELECT
  MAX(active_trains) AS min_platforms_required
FROM running_sum
```
- The maximum number of active trains at any given time equals the minimum number of platforms required.



## Summary of Logic
1. Each train arrival increments the platform demand by 1.
2. Each departure frees the platform the minute after it departs.
3. Merge all events into a timeline and sort them.
4. Track how many trains are active over time.
5. The maximum concurrent trains is your answer.